### This is an extended version of the previous code. Users can specify all the movies they want info about in the movies list given in the below cell. This block of code will not only print the information simultaneously but you can also get the data in tabular form also.

In [45]:
movies= ['Audrie & Daisy','Bitcoin Heist 2016','TVF Pitchers','The godfather']


from bs4 import BeautifulSoup
import requests

titlelist=[]
typelist=[]
durationlist=[]
imdb_ratinglist=[]
votelist=[]
metascorelist=[]
castlist=[]
directorlist=[]
countrylist=[]
languagelist=[]
rdatelist=[]
genrelist=[]
certificatelist=[] 
storylist=[]



for input in movies:
    response = requests.get('http://www.imdb.com/find?ref_=nv_sr_fn&q=' +input+ ' &s=tt')
    soup = BeautifulSoup(response.content, 'lxml')
    # Fetching the movie id so that the browser can be directed to that webpage. Note out of many searches we are taking the 1st search result
    try:
        soup = BeautifulSoup(response.content, 'lxml')
        table = soup.find('table',class_='findList')
        movieid = table.tr.a['href']
    except Exception as e:
        continue
    movielink = "http://www.imdb.com" + movieid
    moviepage = requests.get(movielink)
    soup2 = BeautifulSoup(moviepage.content, 'lxml')
    
    print('\n')
    print('\n')
    print('\n')
    print('\n')
    
    #title of movie/show
    uncleaned = soup2.find('div', class_ = 'title_wrapper').h1.text
    #it contains some weird text and extra &nbsp at end
    uncleaned =uncleaned.replace('\xa0','')
    title = uncleaned.strip()
    print("Title is - {}".format(title))
    titlelist.append(title)

    # Type, Duration/Seasons
    durncounter=1
    wrapper = soup2.select_one('.title_wrapper .subtext')
    links = wrapper.find_all('a')
    links
    for i in range(0,len(links)):
        if 'releaseinfo' in links[i]['href']:
            value = links[i].string.strip()
    try:
        
        if (value.startswith('TV ') and (not value.startswith('TV Special'))): 
            type = 'TV Series'
            typelist.append(type)
            seasons = soup2.select('.seasons-and-year-nav a')[0].string
            duration = seasons+" seasons"
        else:
            type='Movie'
            typelist.append(type)
            try:
                duration = wrapper.find('time').string.strip()
            except Exception as e:
                durncounter=0
                
        if(type == 'Movie') and durncounter!= 0:
            print("\nThe duration is", duration)
            durationlist.append(duration)
        elif(type == 'TV Series'):
            print("\nThe number of seasons are ", duration)
            durationlist.append(duration)
        else:
            print("\nDuration is not given on imdb page")
            durationlist.append(duration)
            
    except Exception as e:
        print("\nDuration is not given on imdb page")
        durationlist.append(None)
        pass

    #imdb rating
    imdbcounter=1
    try:
        imdb_rating = float(soup2.select('.ratingValue span')[0].text)
    except Exception as e:
        imdbcounter=0
    if(imdbcounter !=0):
        print("\nThe imdb rating of the {} is = {}".format(type,imdb_rating))
        imdb_ratinglist.append(imdb_rating)
    else:
        print("\nimdb rating is not given on imdb page")
        imdb_ratinglist.append(imdb_rating)



    #Total number of votes
    votecounter=1
    try:
        votes = soup2.find('span', {'itemprop':'ratingCount'}).string
        votes = votes.replace(',',"")
        votes = int(votes)
    except Exception as e:
        votecounter=0
    if(votecounter!=0):
        print("\nThe imdb rating is calculating on the basis of {} mumber of votes".format( votes))
        votelist.append(votes)
    else:
        print("\nNumber of votes are not given on imdb page")
        votelist.append(votes)

    #metascore    
    try:
        metascore = float(soup2.select('.metacriticScore span')[0].text) if float(soup2.select('.metacriticScore span')[0].text) else None
        print("\nMetascore - ",metascore)
        metascorelist.append(metascore)
    except Exception as e:
        metascore=None
        print("\nMetascore is not available in imdb site")
        metascorelist.append(metascore)

    # cast values
    namelist=[]
    oddnames=[]
    evennames =[]
    ttag = soup2.find_all('tr', class_= 'odd')
    for i in ttag:
        namelist.append( i.find('td', attrs={'class': None}))

    for i in namelist:
        if (i.a) is None:
            pass
        else:
            oddnames.append(i.a.string.strip())

    ttag = soup2.find_all('tr', class_= 'even')
    namelist=[]

    for i in ttag:
        namelist.append( i.find('td', attrs={'class': None}))

    for i in namelist:
        if (i.a) is None:
            pass
        else:
            evennames.append(i.a.string.strip())

    cast = oddnames+evennames
    print("\nCast is as follows",cast)
    castlist.append(cast)

    # Extracting directors
    directors=[]
    director_container = soup2.find('div',class_ = 'credit_summary_item')

    if(director_container is None):
        print("\nDirector information not given in imdb site")
        directors.append(None)
        directorlist.append(directors)
        pass
    elif((director_container.find('h4').string =='Director:') or (director_container.find('h4').string =='Creator:') or 
         (director_container.find('h4').string =='Creators:') or (director_container.find('h4').string =='Directors:')) :
        directors_container = director_container.find_all('a')
        for i in directors_container:
            directors.append(i.string)
        directorlist.append(directors)
        print("\nDirectors are", directors)
    
            
    # Country of production
    country=[]
    titledetails = soup2.find('div',id='titleDetails')
    titleheading = titledetails.find_all('a')
    for i in range(0,len(titleheading)):
        if('country_of_origin' in titleheading[i]['href']):
            country.append(titleheading[i].string)
    print("\nProduction Countries-", country)
    countrylist.append(country)

    # Language
    languages=[]
    titledetails2 = soup2.find('div',id='titleDetails')
    titleheading2 = titledetails2.find_all('a')
    for i in range(0,len(titleheading2)):
        if('primary_language' in titleheading2[i]['href']):
            languages.append(titleheading2[i].string)
    print("\n{} is available in Languages- {}".format(type, languages))
    languagelist.append(languages)

    # release date
    
    try:
        dateheading = soup2.select('.txt-block h4')
        for i in range(0,len(dateheading)):
            if(dateheading[i].string=='Release Date:'):
                releasedate = dateheading[i].next_sibling.strip()
        print("\n{} was released in- {}".format(type, releasedate))
        rdatelist.append(releasedate)
    except Exception as e:
        print("Release info not available on imdb")
        rdatelist.append(None)
        pass


    # genres
    genres=[]
    wrapper = soup2.select_one('.title_wrapper .subtext')
    links = wrapper.find_all('a')
    for i in links:
        if 'genres' in i['href']:
            genres.append(i.string)
    print("\nFollowing are the genres of the {} - {}".format(type,genres))
    genrelist.append(genres)

    # Certificate
    wrapper = soup2.select_one('.title_wrapper .subtext')
    certificate = wrapper.next_element.strip()
    if(certificate != ''):
        print("\nThe {} has been given {} certificate".format(type,certificate))
        certificatelist.append(certificate)
    else:
        print("\nCertificate details are not given on the imdb page")
        certificatelist.append(certificate)


    # Storyline
    # Here we have links inside story for some movies. So normal .strings wont work
    story = soup2.select_one('#titleStoryLine div span').get_text().strip()
    if(story!=''):
        print("\nHere is a description of the storyline - \n", story)
        storylist.append(story)
    else:
        print("\nStory description is not given on imdb page")
        storylist.append(story)











Title is - Audrie

Duration is not given on imdb page

imdb rating is not given on imdb page

Number of votes are not given on imdb page

Metascore is not available in imdb site

Cast is as follows []

Directors are ['Jake Foy']

Production Countries- []

TV Series is available in Languages- []

TV Series was released in- 24 March 1972 (USA)

Following are the genres of the TV Series - ['Musical']

Certificate details are not given on the imdb page

Here is a description of the storyline - 
 Add content advisory for parents »








Title is - Bitcoin Heist(2016)

The duration is 1h 50min

The imdb rating of the Movie is = 5.8

The imdb rating is calculating on the basis of 327 mumber of votes

Metascore is not available in imdb site

Cast is as follows ['Kate Nhung', 'Petey Majik Nguyen', 'Jayvee Mai The Hiep', 'Van Veronica Ngo', 'Thanh Pham', 'Suboi', 'Lam Thanh My', 'Teo Yoo']

Directors are ['Ham Tran']

Production Countries- ['Vietnam']

Movie is available in Languages- 

In [46]:
import pandas as pd


d= {'title':titlelist,'type':typelist,'duration':durationlist,'imdb_rating':imdb_ratinglist,'votelist':votes,
    'metascore':metascorelist,'cast':castlist,'director':directorlist,'country':countrylist,'language':languagelist,
    'rdate':rdatelist,'genre':genrelist,'certificate':certificatelist,'story':storylist}

results= pd.DataFrame(d)
results

,title,type,duration,imdb_rating,votelist,metascore,cast,director,country,language,rdate,genre,certificate,story
0,Audrie,TV Series,None,9.2,1556546,NaN,[],[Jake Foy],[],[],24 March 1972 (USA),[Musical],,Add content advisory for parents »
1,Bitcoin Heist(2016),Movie,1h 50min,5.8,1556546,NaN,"[Kate Nhung, Petey Majik Nguyen, Jayvee Mai Th...",[Ham Tran],[Vietnam],"[Vietnamese, English, German]",4 February 2016 (Vietnam),"[Action, Comedy]",,Hackers have become the bank robbers in the ne...
2,TVF Pitchers,TV Series,1 seasons,9.2,1556546,NaN,"[Naveen Kasturia, Jitendra Kumar, Akanksha Tha...",[Arunabh Kumar],[India],"[Hindi, English]",3 June 2015 (India),"[Comedy, Drama]",,"Four friends Naveen Bansal (Naveen), Jitendra ..."
3,The Godfather(1972),Movie,2h 55min,9.2,1556546,100.0,"[Marlon Brando, James Caan, Robert Duvall, Joh...",[Francis Ford Coppola],[USA],"[English, Italian, Latin]",24 March 1972 (USA),"[Crime, Drama]",A,"The Godfather ""Don"" Vito Corleone is the head ..."


In [27]:
results.to_csv('movies.csv')